In [16]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from sklearn.neighbors import KDTree

from scipy.spatial import KDTree
from shapely.geometry import Point

DATA_DIR = '../data/raw/'
INTERIM_DIR = '../data/interim/'
PROCESSED_DIR = '../data/processed/' 

In [17]:
df_pin = pd.read_csv('../data/interim/universe_pin.csv')
df_pin['pin10'] = df_pin['pin10'].astype(int)
df_pin

,pin10,lon,lat
0,2503106015,-87.623728,41.733446
1,2510117011,-87.616102,41.717261
2,2022402038,-87.610924,41.771698
3,2034101010,-87.622505,41.750465
4,2502207033,-87.587231,41.735210
...,...,...,...
548502,1909325017,-87.760753,41.793282
548503,1330205028,-87.789191,41.936690
548504,1334121012,-87.738512,41.919665
548505,1307225013,-87.793550,41.977342


In [3]:
df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))
df_sales

/var/folders/fk/9chv4jr11ln50p4dhsxntpcw0000gn/T/ipykernel_5978/3652251578.py:1: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(os.path.join(DATA_DIR, 'sales_data.csv'))


,pin,year,township_code,nbhd,class,sale_date,is_mydec_date,sale_price,doc_no,deed_type,mydec_deed_type,seller_name,is_multisale,num_parcels_sale,buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id
0,31012140340000,2000.0,32,32050,278,2000-04-01T00:00:00.000,False,177500.0,317676,Trustee,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,96289215
1,14291030261014,2000.0,73,73150,299,2000-02-01T00:00:00.000,False,315000.0,326770,Warranty,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,97142869
2,13164060180000,2000.0,71,71101,203,2000-06-01T00:00:00.000,False,192000.0,519440,Warranty,NaN,NaN,False,1.0,NaN,LAND AND BUILDING,False,False,False,96585660
3,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,NaN,US BANK,False,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542
4,19354040600000,2016.0,72,72200,205,2016-08-01T00:00:00.000,False,1.0,1625129009,Warranty,NaN,NaN,True,5.0,NaN,LAND AND BUILDING,True,True,False,98154864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615947,13154100321010,2025.0,71,71074,299,2025-01-29T00:00:00.000,True,250000.0,2502930093,Warranty,Warranty Deed,MARITZA FELICIANO,False,1.0,WANDA CORNIER,NaN,False,False,False,7699628
2615948,13224170200000,2025.0,71,71070,203,2025-01-20T00:00:00.000,True,399900.0,2502922033,Warranty,Warranty Deed,RAYMOND E ROHDE,False,1.0,JUAN FLORES,NaN,False,False,False,7699788
2615949,13262110390000,2025.0,71,71070,205,2025-01-16T00:00:00.000,True,1120000.0,2502824103,Warranty,Warranty Deed,BENJAMIN KREAM,False,1.0,ELIZABETH P. ASTRUP,NaN,False,False,False,7699553
2615950,14052150151139,2025.0,73,73031,299,2025-05-01T00:00:00.000,True,470000.0,2513622266,Warranty,Warranty Deed,GARY BELSON,False,1.0,JULIE RACKOW,NaN,False,False,False,7727049


In [4]:
df_sales['pin10'] = df_sales['pin'].astype(str).str[:10].astype(int)
df_sales.dtypes

pin                                   int64
year                                float64
township_code                         int64
nbhd                                  int64
class                                object
sale_date                            object
is_mydec_date                          bool
sale_price                          float64
doc_no                               object
deed_type                            object
mydec_deed_type                      object
seller_name                          object
is_multisale                           bool
num_parcels_sale                    float64
buyer_name                           object
sale_type                            object
sale_filter_same_sale_within_365       bool
sale_filter_less_than_10k              bool
sale_filter_deed_type                  bool
row_id                                int64
pin10                                 int64
dtype: object

In [5]:
df_merged = pd.merge(df_sales, df_pin, on='pin10', how='left')
df_merged

,pin,year,township_code,nbhd,class,sale_date,is_mydec_date,sale_price,doc_no,deed_type,...,num_parcels_sale,buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id,pin10,lon,lat
0,31012140340000,2000.0,32,32050,278,2000-04-01T00:00:00.000,False,177500.0,317676,Trustee,...,1.0,NaN,LAND AND BUILDING,False,False,False,96289215,3101214034,-87.677174,41.551951
1,14291030261014,2000.0,73,73150,299,2000-02-01T00:00:00.000,False,315000.0,326770,Warranty,...,1.0,NaN,LAND AND BUILDING,False,False,False,97142869,1429103026,-87.660681,41.939158
2,13164060180000,2000.0,71,71101,203,2000-06-01T00:00:00.000,False,192000.0,519440,Warranty,...,1.0,NaN,LAND AND BUILDING,False,False,False,96585660,1316406018,-87.748956,41.959141
3,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,...,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542,2423302037,-87.714163,41.683795
4,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,...,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542,2423302037,-87.714163,41.683795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626628,13224170200000,2025.0,71,71070,203,2025-01-20T00:00:00.000,True,399900.0,2502922033,Warranty,...,1.0,JUAN FLORES,NaN,False,False,False,7699788,1322417020,-87.729893,41.943715
2626629,13262110390000,2025.0,71,71070,205,2025-01-16T00:00:00.000,True,1120000.0,2502824103,Warranty,...,1.0,ELIZABETH P. ASTRUP,NaN,False,False,False,7699553,1326211039,-87.710318,41.936140
2626630,14052150151139,2025.0,73,73031,299,2025-05-01T00:00:00.000,True,470000.0,2513622266,Warranty,...,1.0,JULIE RACKOW,NaN,False,False,False,7727049,1405215015,-87.654796,41.991306
2626631,14191230381002,2025.0,73,73070,299,2025-04-11T00:00:00.000,True,590000.0,2511824094,Warranty,...,1.0,EMILY CLYSDALE,NaN,False,False,False,7718370,1419123038,-87.678963,41.948944


In [6]:
df_merged.columns

Index(['pin', 'year', 'township_code', 'nbhd', 'class', 'sale_date',
       'is_mydec_date', 'sale_price', 'doc_no', 'deed_type', 'mydec_deed_type',
       'seller_name', 'is_multisale', 'num_parcels_sale', 'buyer_name',
       'sale_type', 'sale_filter_same_sale_within_365',
       'sale_filter_less_than_10k', 'sale_filter_deed_type', 'row_id', 'pin10',
       'lon', 'lat'],
      dtype='object')

In [7]:
# Drop all rows with missing lon/lat
df_merged = df_merged.dropna(subset=['lon', 'lat'])
df_merged

,pin,year,township_code,nbhd,class,sale_date,is_mydec_date,sale_price,doc_no,deed_type,...,num_parcels_sale,buyer_name,sale_type,sale_filter_same_sale_within_365,sale_filter_less_than_10k,sale_filter_deed_type,row_id,pin10,lon,lat
0,31012140340000,2000.0,32,32050,278,2000-04-01T00:00:00.000,False,177500.0,317676,Trustee,...,1.0,NaN,LAND AND BUILDING,False,False,False,96289215,3101214034,-87.677174,41.551951
1,14291030261014,2000.0,73,73150,299,2000-02-01T00:00:00.000,False,315000.0,326770,Warranty,...,1.0,NaN,LAND AND BUILDING,False,False,False,97142869,1429103026,-87.660681,41.939158
2,13164060180000,2000.0,71,71101,203,2000-06-01T00:00:00.000,False,192000.0,519440,Warranty,...,1.0,NaN,LAND AND BUILDING,False,False,False,96585660,1316406018,-87.748956,41.959141
3,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,...,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542,2423302037,-87.714163,41.683795
4,24233020370000,2014.0,39,39250,100,2014-06-01T00:00:00.000,False,500.0,1427529079,Other,...,1.0,MY OWN DOMINION LP,LAND,False,True,False,96639542,2423302037,-87.714163,41.683795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2626628,13224170200000,2025.0,71,71070,203,2025-01-20T00:00:00.000,True,399900.0,2502922033,Warranty,...,1.0,JUAN FLORES,NaN,False,False,False,7699788,1322417020,-87.729893,41.943715
2626629,13262110390000,2025.0,71,71070,205,2025-01-16T00:00:00.000,True,1120000.0,2502824103,Warranty,...,1.0,ELIZABETH P. ASTRUP,NaN,False,False,False,7699553,1326211039,-87.710318,41.936140
2626630,14052150151139,2025.0,73,73031,299,2025-05-01T00:00:00.000,True,470000.0,2513622266,Warranty,...,1.0,JULIE RACKOW,NaN,False,False,False,7727049,1405215015,-87.654796,41.991306
2626631,14191230381002,2025.0,73,73070,299,2025-04-11T00:00:00.000,True,590000.0,2511824094,Warranty,...,1.0,EMILY CLYSDALE,NaN,False,False,False,7718370,1419123038,-87.678963,41.948944


In [8]:
CTA_FILE_PATH = os.path.join(DATA_DIR, 'cta_l_stops.geojson')
gdf_cta = gpd.read_file(CTA_FILE_PATH)
gdf_cta.to_crs(epsg=4326, inplace=True)
gdf_cta.head()

,station_id,point_y,pknrd,legend,point_x,address,ada,longname,lines,geometry
0,970,1896352.78926296,False,Blue Line,1144440.97667316,720 S. Cicero Avenue,False,Cicero-Congress,Blue Line (Congress),POINT (-87.74517 41.87161)
1,20,1901803.39560403,False,Green Line,1128608.76033842,1 S. Harlem Avenue,True,Harlem-Lake,Green Line (Lake),POINT (-87.80318 41.88685)
2,610,1901950.19138017,False,Green Line,1133921.86037537,36 N. Ridgeland Avenue,False,Ridgeland,Green Line (Lake),POINT (-87.78366 41.88716)
3,230,1937256.04587705,True,Blue Line,1118914.13069739,5800 N. Cumberland Avenue,True,Cumberland,Blue Line,POINT (-87.83803 41.98429)
4,1700,1900828.68763513,False,Multiple Lines,1176812.47724731,29 N. Wabash,True,Washington/Wabash,"Brown, Orange, Pink, Purple (Express), Green",POINT (-87.62619 41.88322)


In [9]:
# Create Point objects from lat/lon columns
geometry = [Point(xy) for xy in zip(df_merged['lon'], df_merged['lat'])]

In [10]:
# Create a GeoDataFrame for sales data
gdf_properties = gpd.GeoDataFrame(
    df_merged,
    geometry=geometry,
    crs="EPSG:4326"
)
print(f"Properties converted to GeoDataFrame. Shape: {gdf_properties.shape}")

Properties converted to GeoDataFrame. Shape: (1782572, 24)


In [11]:
# Convert CTA coordinates (Lon/Lat) to a 2D array
cta_coordinates = np.array(list(zip(gdf_cta.geometry.x, gdf_cta.geometry.y)))
# Convert property coordinates to a 2D array
property_coordinates = np.array(list(zip(gdf_properties.geometry.x, gdf_properties.geometry.y)))

In [12]:
# KDTree for fast nearest neighbor searching
tree = KDTree(cta_coordinates)
# Query the KDTree
distances, indices = tree.query(property_coordinates, k=1)

In [13]:
gdf_properties['min_distance_deg'] = distances
nearest_stations = gdf_cta.iloc[indices].reset_index(drop=True)
gdf_properties['nearest_cta_stop'] = nearest_stations['longname']
gdf_properties['nearest_cta_lines'] = nearest_stations['lines']

In [14]:
M_PER_DEGREE = 111111 
gdf_properties['min_distance_meters'] = gdf_properties['min_distance_deg'] * M_PER_DEGREE
gdf_properties[['pin', 'sale_price', 'min_distance_meters', 'nearest_cta_stop', 'nearest_cta_lines']].head()

,pin,sale_price,min_distance_meters,nearest_cta_stop,nearest_cta_lines
0,31012140340000,177500.0,19822.744450,95th/Dan Ryan,Red Line
1,14291030261014,315000.0,610.954725,Southport,Brown Line
2,13164060180000,192000.0,647.953208,Montrose-O'Hare,Blue Line
3,24233020370000,500.0,10854.339658,95th/Dan Ryan,Red Line
4,24233020370000,500.0,10854.339658,95th/Dan Ryan,Red Line


In [15]:
df_final = pd.DataFrame(gdf_properties.drop(columns=['geometry', 'min_distance_deg']))
df_final.to_csv(os.path.join(INTERIM_DIR, 'sales_data_enriched.csv'), index=False)